<a href="https://colab.research.google.com/github/valentinaflores/SDistribuidos/blob/main/producto_mv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install mpi4py

In [38]:
from mpi4py import MPI
import numpy as np
import time

In [39]:
# Inicializar el entorno MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

In [40]:
# Dimensiones de la matriz y el vector
matrix_rows = 3
matrix_cols = 3
vector_size = 3


In [41]:
# Generar matriz y vector aleatorios en el proceso raíz
if rank == 0:
    matrix = np.array([[3.0, 3.0, 3.0], [3.0, 3.0, 3.0], [3.0, 3.0, 3.0]])
    vector = np.array([2.0, 2.0, 2.0])
else:
    matrix = None
    vector = None

In [42]:
# Distribuir la matriz y el vector a todos los procesos
matrix = comm.bcast(matrix, root=0)
vector = comm.bcast(vector, root=0)

In [52]:
# Medir el tiempo inicial
start_time = time.time()

In [53]:
# Calcular la porción de la matriz y el vector para cada proceso
local_matrix = np.zeros((matrix_rows // size, matrix_cols))
local_vector = np.zeros(vector_size)
comm.Scatter(matrix, local_matrix, root=0)
comm.Scatter(vector, local_vector, root=0)

In [54]:
# Realizar la multiplicación local del vector por la matriz
local_result = np.dot(local_matrix, local_vector)

In [55]:
# Realizar la reducción de suma para obtener el resultado final
result = np.zeros(matrix_rows)
comm.Allreduce(local_result, result, op=MPI.SUM)

In [56]:
# Medir el tiempo final
end_time = time.time()

In [57]:
# Imprimir el resultado en el proceso raíz
if rank == 0:
    print("Resultado de la multiplicación:")
    print(result)

Resultado de la multiplicación:
[18. 18. 18.]


In [58]:
# Calcular el tiempo transcurrido
elapsed_time = end_time - start_time
print(f"Tiempo transcurrido: {elapsed_time} segundos")

Tiempo transcurrido: 6.183919906616211 segundos
